<a href="https://colab.research.google.com/github/Kimhansav/everynocode_search_engine/blob/main/BP_judge_question.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#한국 버블 커뮤니티 오픈톡방 대화 중 질문을 선별하는 코드(로컬)

In [ ]:
!pip install datasets
!pip install kss
!pip install accelerate -U
import accelerate
import kss
import pandas as pd
import numpy as np
import re
import os
import tensorflow as tf
import urllib.request
import torch
import torch.nn as nn
from transformers.modeling_outputs import SequenceClassifierOutput
from tqdm import tqdm
from torch.nn.functional import cross_entropy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification, TextClassificationPipeline, Trainer, TrainingArguments, RobertaConfig, shape_list, BertTokenizer, TFBertModel, RobertaModel, RobertaTokenizer, TrainerCallback
from datasets import Dataset, load_dataset
from sklearn.model_selection import StratifiedKFold, train_test_split
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 모델과 토크나이저 초기화
HUGGINGFACE_MODEL_PATH = "bespin-global/klue-roberta-small-3i4k-intent-classification"
tokenizer = RobertaTokenizerFast.from_pretrained(HUGGINGFACE_MODEL_PATH)
pretrained_model = RobertaForSequenceClassification.from_pretrained(HUGGINGFACE_MODEL_PATH)

In [ ]:
#출력 레이블 개수를 바꾸기 위해 출력층 하나 더 쌓기
class ExtendedRobertaForSequenceClassification(nn.Module):
    def __init__(self, pretrained_model):
        super(ExtendedRobertaForSequenceClassification, self).__init__()
        # 기존 파인튜닝 모델 로드
        self.pretrained_model = pretrained_model
        self.config = RobertaConfig(
            _name_or_path = "BP_judge_question",
            architectures = ["RobertaForSequenceClassification"],
            attention_probs_dropout_prob = 0.1,
            bos_token_id = 0,
            eos_token_id = 2,
            hidden_act = "gelu",
            hidden_dropout_prob = 0.1,
            hidden_size = 768,
            id2label = {"0": "not-question", "1": "question"},
            initializer_range = 0.02,
            intermediate_size = 3072,
            label2id = {"question": 1, "not-question": 0},
            layer_norm_eps = 1e-05,
            max_position_embeddings = 514,
            model_type = "roberta",
            num_attention_heads = 12,
            num_hidden_layers = 6,
            pad_token_id = 1,
            position_embedding_type = "absolute",
            tokenizer_class = "BertTokenizer",
            transformers_version = "4.20.0",
            type_vocab_size = 1,
            use_cache = True,
            vocab_size = 32000,
            num_labels = 2
        )  # 기존 모델의 config를 수정 후 저장

        # 새 태스크를 위한 추가 출력층, 7개 출력을 받아서 2개로 분류
        self.additional_classifier = nn.Linear(7, 2)

    #AttributeError: 'ExtendedRobertaForSequenceClassification' object has no attribute 'can_generate' 해결용 함수. 이유 아직 모름
    def can_generate(self):
        return False  # 메서드로 변경

    def forward(self, input_ids, attention_mask=None, labels = None):
        # 기존 모델을 통해 입력 데이터 처리
        outputs = self.pretrained_model(input_ids, attention_mask=attention_mask)

        #Softmax 층 통과
        #일단 없이 사용
        # probs = torch.softmax(logits, dim = -1)

        # 새로운 태스크의 예측을 위한 추가 출력층 사용
        logits = self.additional_classifier(outputs.logits)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.config.num_labels), labels.view(-1))

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions
        )

# 확장된 모델 인스턴스 생성
model = ExtendedRobertaForSequenceClassification(pretrained_model)

In [ ]:
model = pretrained_model
#GPU 사용 코드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#파이프라인을 사용하는 경우 GPU로 모델을 옮겨준 뒤 파이프라인에 모델을 넣어줘야 함
model.to(device)

In [ ]:
#전처리된 세 파일을 데이터프레임으로 로드
df_qna = pd.read_excel('/content/drive/My Drive/community_qna_preprocessed_spaced.xlsx')
df_all_contents = pd.read_excel('/content/drive/My Drive/community_all_contents_preprocessed_spaced.xlsx')
df_all_comments = pd.read_excel('/content/drive/My Drive/community_all_comments_preprocessed_spaced.xlsx')

왜 빌더로그 게시글 요소들을 세 부분으로 나눠야 하는가?
1. positive sample, negative sample 비율 1대1이 되도록 만들기
2. 사용할 RobertaForSequenceClassification 아키텍처를 보면 (position_embeddings): Embedding(514, 768, padding_idx=1)이 포함되어 있다. negative sample에 해당하는 빌더로그 게시글만 길이가 길다면 이러한 정보까지 학습할 여지가 있다.

왜 tokenizer()로 바로 시퀀스 길이를 구하지 않고 원본 리스트부터 시작해서 평균 길이를 구하는가
->

인자 중 max_length가 존재하기 때문에 어차피 길이를 초과하는 데이터는 truncation이 진행된다.

그렇기 때문에 데이터 평균 길이를 비교하려면 원본 리스트를 사용해야 한다.

In [ ]:
#데이터셋의 tokenized data의 길이를 구하는 함수
def tokenized_sequence_length(text):
  #kss 모듈로 글을 문장으로 분리
  original_sentences = kss.split_sentences(text)
  index_sentences = tokenizer(original_sentences)['input_ids']
  index_sentences_filtered = [sentence[1:-1] for sentence in index_sentences] #cls_token과 sep_token 제외
  total_length = sum(len(s) for s in index_sentences_filtered)
  return index_sentences_filtered, total_length

In [ ]:
#데이터셋에서 각 요소를 n등분하는 함수
def divide_text_to_number(text, number):
  #kss 모듈로 문장으로 분리
  index_sentences, total_length = tokenized_sequence_length(text)
  target_length = total_length / number

  # 세 부분으로 분할
  segments = []
  current_segment = []
  current_length = 0

  #마지막 세그먼트 추가를 위해 사용하는 인덱스 값
  sentence_checkpoint = 0

  for index, sentence in enumerate(index_sentences):
      if current_length + len(sentence) > target_length and current_segment:
          segments.append(sum(current_segment, []))
          current_segment = [sentence]
          current_length = len(sentence)
          if len(segments) == 2:  # 이미 두 세그먼트를 만들었다면 나머지는 모두 마지막 세그먼트에 추가
              sentence_checkpoint = index #current_segment에 추가해둔 문장을 인덱싱
              break
      else:
          current_segment.append(sentence)
          current_length += len(sentence)

  # 마지막 세그먼트 추가
  segments.append(sum(current_segment + index_sentences[sentence_checkpoint + 1:], []))

  return tokenizer.batch_decode(segments)


In [ ]:
#BP_judge_question용 데이터 제작

#slug가 있다면 질문, 없다면 답변, 이 중 질문을 positive sample로 설정
question_list = df_qna[pd.isna(df_qna['Slug']) == False]['내용'].dropna(axis = 0).to_list()
label_question = [1] * len(question_list) #모델 설

print(question_list)
print(len(question_list))

#질문 평균 길이 확인(원본 텍스트)
print('원본 텍스트 평균 길이 :', sum(len(question) for question in question_list) / len(question_list))

#질문 평균 길이 확인(토큰화 결과)
print('토큰화된 텍스트 평균 길이 :', sum(tokenized_sequence_length(text)[1] for text in question_list) / len(question_list))

In [ ]:
#답변을 negative sample로 설정
answer_list = df_qna['답변'].dropna(axis = 0).to_list()
label_answer = [0] * len(answer_list)

print(answer_list)
print(len(answer_list))

#답변 평균 길이 확인(원본 텍스트)
print('원본 텍스트 평균 길이 :', sum(len(answer) for answer in answer_list) / len(answer_list))

#답변 평균 길이 확인(토큰화 결과)
print('토큰화된 텍스트 평균 길이 :', sum(tokenized_sequence_length(text)[1] for text in answer_list) / len(answer_list))

In [ ]:
#빌더로그 글을 negative sample로 설정
builder_log_list = df_all_contents[df_all_contents['커뮤니티 타입'] == '빌더 로그']['내용'].dropna(axis = 0).to_list()

print(builder_log_list)
print(len(builder_log_list))

#빌더로그 글 평균 길이 확인(원본 텍스트)
print('원본 텍스트 평균 길이 :', sum(len(builder_log) for builder_log in builder_log_list) / len(builder_log_list))

#빌더로그 글 평균 길이 확인(토큰화 결과)
print('토큰화된 텍스트 평균 길이 :', sum(tokenized_sequence_length(text)[1] for text in builder_log_list) / len(builder_log_list))

#시퀀스 길이 균형을 위해 각 빌더로그 글을 3등분
builder_log_list_divided = sum([divide_text_to_number(text, 3) for text in builder_log_list], [])

#분할한 데이터의 개수에 맞춰서 레이블 설정
label_builder_log = [0] * (len(builder_log_list_divided))

#분리된 빌더로그 글 평균 길이 확인(토큰화 결과)
print('분리된 데이터셋에서 토큰화된 텍스트 평균 길이 :', sum(tokenized_sequence_length(text)[1] for text in builder_log_list_divided) / len(builder_log_list_divided))

In [ ]:
#데이터프레임 생성
text = question_list + answer_list + builder_log_list_divided
label = label_question + label_answer + label_builder_log

df_judge_question = pd.DataFrame([text, label], index = ['text', 'label'])
df_judge_question = df_judge_question.T
print(df_judge_question)

In [ ]:
#훈련(훈련+검증)/테스트 9:1 분리
X, y = df_judge_question["text"], df_judge_question["label"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, shuffle = True, stratify = y) #초기에는 정렬된 데이터셋으로 stratify로 레이블 비율을 유지하며 데이터셋을 섞은 후 분리

#pandas dataframe으로 변환
train_df = pd.DataFrame({"text": X_train, "label": y_train})
test_df = pd.DataFrame({"text": X_test, "label": y_test})

# Hugging Face의 Dataset 객체로 변환
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# 데이터셋 전처리 함수 및 처리
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True, max_length=512)

tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/612 [00:00<?, ? examples/s]

In [ ]:
#과적합 방지를 위한 EarlyStopping 클래스
class EarlyStoppingCallback(TrainerCallback):
    def __init__(self, patience=3):
        self.patience = patience
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def on_evaluate(self, args, state, control, **kwargs):
        # 현재 검증 손실 가져오기
        current_loss = kwargs['metrics']['eval_loss']

        # 최고 성능 갱신 또는 카운터 증가
        if self.best_score is None:
            self.best_score = current_loss
        elif current_loss > self.best_score:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
                control.should_training_stop = True
        else:
            self.best_score = current_loss
            self.counter = 0

In [ ]:
#분류 문제이며 레이블 간 데이터 수 비율을 고려하기 위해 StratifiedKFold 사용

# StratifiedKFold 설정
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
for fold, (train_idx, val_idx) in enumerate(skf.split(tokenized_train_dataset, tokenized_train_dataset['label'])):

    # 훈련 세트와 검증 세트 분리
    train_dataset = tokenized_train_dataset.select(train_idx)
    val_dataset = tokenized_train_dataset.select(val_idx)

    # 훈련 설정
    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy = "steps",
        eval_steps = 500,
        save_strategy = "steps",
        save_steps = 500,
        learning_rate = 2e-5,
        num_train_epochs = 3, #고려사항
        per_device_train_batch_size = 8,
        per_device_eval_batch_size = 8,
        warmup_steps = 100, #고려사항
        weight_decay = 0.01, #고려사항
        logging_dir = "./logs",

        #
    )

    # 트레이너 초기화 및 훈련
    #RobertaForSequenceClassification 모델은 기본적으로 분류 문제에 적합한 loss function을 내장하고 있습니다. num_labels=2로 설정함으로써, 이 모델은 이진 분류 문제를 위해 사전 구성되며, 내부적으로 Cross-Entropy Loss를 사용합니다.
    trainer = Trainer(
        model = model,
        args = training_args,
        train_dataset = train_dataset,
        eval_dataset = val_dataset,
        callbacks = [EarlyStoppingCallback(patience = 2)]
    )

    trainer.train()

In [ ]:
#테스트 데이터셋 준비
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)

#테스트 데이터셋으로 모델 평가
trainer.evaluate(eval_dataset = tokenized_test_dataset)

In [4]:
#카카오톡 대화내용 불러오고 데이터프레임으로 받기
file_path = '/content/drive/My Drive/talk_preprocess_result_short_spaced.xlsx'

df = pd.read_excel(file_path)

In [5]:
#데이터 처리 Pipeline 설정, top_k를 1로 설정시 최대로 측정된 label만 출력됨.
#텍스트 처리해보니 시퀀스 길이가 539인 행 존재 -> truncation 진행, 최대 길이는 512
text_classifier = TextClassificationPipeline(
    tokenizer=tokenizer,
    model=model,
    top_k = 1,
    truncation = True,
    batch_size = 128,
    device = device
)

In [ ]:
# #결측치 제거
# df['text'] = df['text'].dropna(axis = 0)

In [ ]:
# #결측치 검증
# print(type(text[:5]))
# print(all(isinstance(item, str) for item in text))
# for i, item in enumerate(text):
#   if not isinstance(item, str):
#     print(f"Index {i}: {item} (Type: {type(item)})")

In [ ]:
#결측치 제거
df = df.replace('None', np.nan).dropna(axis = 0)

#모델로 prediction_labels 리스트 생성.
text = df['text'].to_list()

#tqdm으로 진행률 표시
predictions= []
for i in tqdm(range(0, len(text), 128), desc='Classifying'):
    batch = text[i:i+128]
    predictions.extend(text_classifier(batch))

# predictions = text_classifier(text)
prediction_labels = [prediction[0]['label'] for prediction in predictions]

#Dataframe에 새 열로 추가.
df['label'] = prediction_labels

In [ ]:
#.csv 파일로 google drive에 저장
save_path = '/content/drive/My Drive/judge_question_result_short.xlsx'

df.to_excel(save_path)